In [1]:
!pip install xgboost

DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\aiofiles-23.2.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\anyio-3.7.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anaconda3\lib\site-packages\boto3-1.26.105-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\nishchal22330\appdata\local\anac

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import hashlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,classification_report
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
df=pd.read_csv('FraudTrain.csv')

In [6]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [8]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [9]:
df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [10]:
df.duplicated().sum()

0

In [11]:
import pandas as pd
import hashlib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import joblib

def transform_df(X):
            def hash_transaction(transaction):
                hashed = hashlib.sha256(transaction.encode()).hexdigest()
                return hashed
            X['trans_date_trans_time'] = X['trans_date_trans_time'].astype(str)
            X['trans_date_trans_time'] = X['trans_date_trans_time'].str.strip('-')
            X['tran_date'] = X['trans_date_trans_time'].str[:10]
            X['tran_time'] = X['trans_date_trans_time'].str[10:]
            X.drop(columns=['trans_date_trans_time'], inplace=True)
            X['tran_yr'] = (X['tran_date'].str[:4]).astype(int)
            X['tran_m'] = (X['tran_date'].str[5:7]).astype(int)
            X['tran_day'] = (X['tran_date'].str[8:]).astype(int)
            X['tran_hr'] = (X['tran_time'].str[2]).astype(int)
            X['tran_min'] = (X['tran_time'].str[4:5]).astype(int)
            X['tran_sec'] = (X['tran_time'].str[7:]).astype(int)
            X.drop(columns=['tran_time', 'tran_date'], inplace=True)
            columns_to_drop=['merchant', 'first', 'last', 'street', 'city','job']
            X.drop(columns=columns_to_drop, inplace=True)
            X['gender'] = X['gender'].map({'M': 0, 'F': 1})
            one_hot_encoded = pd.get_dummies(X['state'], prefix='state')
            X = pd.concat([X, one_hot_encoded], axis=1)
            X.drop(columns=['state'], inplace=True)
            label_encoder = LabelEncoder()
            label_encoder.fit(X['category'])
            col='category'
            X[f'{col}_label_encoded'] = label_encoder.transform(X[col]).astype(int)
            X.drop(columns=[col], inplace=True)
            X['dob'] = pd.to_datetime(X['dob'])
            X['int_dob'] = X['dob'].astype('int64') // 10**9
            X.drop(columns=['dob'], inplace=True)
            X['hashed_trans_num'] = X['trans_num'].apply(hash_transaction)
            vectorizer = CountVectorizer(analyzer='char')
            X_transformed = vectorizer.fit_transform(X['trans_num'])
            df_tokenized = pd.DataFrame(X_transformed.toarray(), columns=vectorizer.get_feature_names_out())
            X = pd.concat([X, df_tokenized], axis=1)
            X.drop(columns=['trans_num', 'hashed_trans_num'], inplace=True)
            bool_columns = X.select_dtypes(include=bool)
            bool_columns_int = bool_columns.astype(int)
            X.drop(columns=bool_columns.columns, inplace=True)
            X = pd.concat([X, bool_columns_int], axis=1)
            return X
# Example usage
# Assuming df is your DataFrame
transformed_data = transform_df(df)
# Save the pipeline as a pickle file
# joblib.dump(transform_df, 'transformed_pip.pkl')


In [12]:
joblib.dump(transform_df, 'transformed_pip.pkl')

['transformed_pip.pkl']

In [13]:
tf_p=joblib.load('transformed_pip.pkl')

In [29]:
transformed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 87 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1296675 non-null  int64  
 1   cc_num                  1296675 non-null  int64  
 2   amt                     1296675 non-null  float64
 3   gender                  1296675 non-null  int64  
 4   zip                     1296675 non-null  int64  
 5   lat                     1296675 non-null  float64
 6   long                    1296675 non-null  float64
 7   city_pop                1296675 non-null  int64  
 8   unix_time               1296675 non-null  int64  
 9   merch_lat               1296675 non-null  float64
 10  merch_long              1296675 non-null  float64
 11  is_fraud                1296675 non-null  int64  
 12  tran_yr                 1296675 non-null  int32  
 13  tran_m                  1296675 non-null  int32  
 14  tr

In [32]:
X=transformed_data.drop(labels=['is_fraud'],axis=1)
y=transformed_data['is_fraud']

In [14]:
df_test=pd.read_csv('fraudTest.csv')

In [15]:
X_test_scaled=df_test.drop(labels=['is_fraud'],axis=1)
y_test_=df_test['is_fraud']

In [16]:
X_test_scaled=tf_p(X_test_scaled)

In [33]:
print("Number of features in X:", X.shape[1])
print("Number of features in X_test_scaled:", X_test_scaled.shape[1])


Number of features in X: 86
Number of features in X_test_scaled: 85


In [34]:

X_columns = set(X.columns)
X_test_columns = set(X_test_scaled.columns)

missing_features = X_columns - X_test_columns

print("Missing feature(s) in X_test_scaled:", missing_features)


Missing feature(s) in X_test_scaled: {'state_DE'}


In [35]:

for missing_feature in missing_features:
    X_test_scaled[missing_feature] = 0  




In [36]:
X_test_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 86 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              555719 non-null  int64  
 1   cc_num                  555719 non-null  int64  
 2   amt                     555719 non-null  float64
 3   gender                  555719 non-null  int64  
 4   zip                     555719 non-null  int64  
 5   lat                     555719 non-null  float64
 6   long                    555719 non-null  float64
 7   city_pop                555719 non-null  int64  
 8   unix_time               555719 non-null  int64  
 9   merch_lat               555719 non-null  float64
 10  merch_long              555719 non-null  float64
 11  tran_yr                 555719 non-null  int32  
 12  tran_m                  555719 non-null  int32  
 13  tran_day                555719 non-null  int32  
 14  tran_hr             

In [37]:
random_f=joblib.load('Random_forest_clf.pkl')

In [38]:
y_pred2=random_f.predict(X_test_scaled)

In [39]:
print(classification_report(y_pred2,y_test_))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    555679
           1       0.00      0.00      0.00        40

    accuracy                           1.00    555719
   macro avg       0.50      0.50      0.50    555719
weighted avg       1.00      1.00      1.00    555719

